In [1]:
import sys
import argparse
import torch
import torch.nn.functional as F
from torch import nn
from torch import optim
import syft as sy
from syft import workers

Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/home/suparerk/anaconda3/envs/pysyft/lib/python3.7/site-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.15.4.so'



In [2]:
use_cuda = torch.cuda.is_available()
torch.manual_seed(1)
device = torch.device("cuda" if use_cuda else "cpu")
print('ประมวลผลโดยใช้ ' + str(device))

ประมวลผลโดยใช้ cpu


In [3]:
hook = sy.TorchHook(torch)

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 1)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    

# แบบจำลองเป็น Neural Network
model = Net()

# จำลองข้อมูลที่ฝั่ง Aggregator_Node เพื่อใช้สร้าง TrainConfig
# ไม่มีผลต่อการคำนวณ เนื่องข้อมูลอยู่ใน Data_Node
mock_data = torch.zeros(1, 2)

# Serialization: สร้าง Traced_Model เป็น jit เพื่อให้สามารถส่งไปรันที่ Data_Node
traced_model = torch.jit.trace(model, mock_data)

# ใช้ Stochastic Gradient Descent
optimizer = "SGD"

# สร้าง Loss Function เป็น jit เพื่อให้สามารถส่งไปรันที่ Data_Node
@torch.jit.script
def loss_fn(target, pred):
    return ((target.view(pred.shape).float() - pred.float()) ** 2).mean()


# Hyperparameters
batch_size = 4
optimizer_args = {"lr" : 0.01, "weight_decay" : 0.01}
epochs = 1
max_nr_batches = -1
shuffle = True

In [ ]:
# TrainConfig 
train_config = sy.TrainConfig(model=traced_model,
                              loss_fn=loss_fn,
                              optimizer=optimizer,
                              batch_size=batch_size,
                              optimizer_args=optimizer_args,
                              epochs=epochs,
                              shuffle=shuffle)

# Must change the host IP address!!!
kwargs_websocket = {"host": "10.7.3.34", "hook": hook, "verbose": True}
data_node = sy.workers.websocket_client.WebsocketClientWorker(
    id="data_node", 
    port=8777, 
    **kwargs_websocket)

train_config.send(data_node)

for epoch in range(10):
    loss = data_node.fit(dataset_key="welfare")
    print("-" * 50)
    print("Iteration %s: Data_Node's loss: %s" % (epoch, loss))

new_model = train_config.model_ptr.get()

In [5]:
# Testing
import pandas as pd 
import numpy as np 
df = pd.read_csv('test_data/welfare_test_data.csv')
df.head()

,Sex,Age,Occupation,Education,Elderly,Disabled,Chronic_Patient,Self_Reliance,Province_ID,Amphur_ID,...,Accommodation_Doc_Type,Occupation_Own,Occupation_Rent,Occupation_Public,Occupation_Other,Occupation_Other_Name,Occupation_Doc_Type,Happiness,Owner_Income,Label
0,ช,44,ธุรกิจส่วนตัว,ป.ตรี หรือเทียบเท่า,0,0,0,0,73,7303,...,"โฉนดที่ดิน (น.ส.4, น.ส.4 ก, น.ส.4 ข, น.ส.4 ค, ...",5,0,0,0,0,"โฉนดที่ดิน (น.ส.4, น.ส.4 ก, น.ส.4 ข, น.ส.4 ค, ...",8,10200000.0,0
1,ญ,49,เกษตร-ทำนา,"ต่ำกว่าชั้นประถมฯ (ป.4, ป.7, ป.6)",0,0,0,0,44,4408,...,"โฉนดที่ดิน (น.ส.4, น.ส.4 ก, น.ส.4 ข, น.ส.4 ค, ...",0,0,0,0,0,0,8,5464000.0,0
2,ช,41,ค้าขาย,ป.ตรี หรือเทียบเท่า,0,0,0,0,71,7110,...,สิทธิในการทำเกษตรกรรม (ส.ป.ก.4-01),0,0,0,0,0,0,7,2000000.0,0
3,ช,59,ค้าขาย,"ประถมฯ (ป.4, ป.7, ป.6)",0,0,0,0,71,7105,...,0,0,0,0,0,0,0,9,1800000.0,0
4,ญ,46,เกษตร-ทำนา,ม.ปลาย (มศ.4-5 หรือ ม.4-6 หรือ ปวช.),0,0,0,0,71,7106,...,"โฉนดที่ดิน (น.ส.4, น.ส.4 ก, น.ส.4 ข, น.ส.4 ค, ...",5,0,0,0,0,"โฉนดที่ดิน (น.ส.4, น.ส.4 ก, น.ส.4 ข, น.ส.4 ค, ...",7,1700000.0,0


In [6]:
X_Columns = ['Age', 'Owner_Income']
X = df[X_Columns]
X = X.astype(np.float)
X['Age'] = (X['Age'] - 51.178235647129426) / 13.711646221534574
X['Owner_Income'] = (X['Owner_Income'] - 159077.33560521627) / 13.711646221534574
test_data = torch.tensor(X.values, dtype=torch.float32, requires_grad=True)

In [7]:
y_Column = ['Label']
y = df[y_Column]
y = y.astype(np.float)
target = torch.tensor(y.values, dtype=torch.float32, requires_grad=False)

In [8]:
print("\nEvaluation after training:")
pred = new_model(test_data)
prediction = torch.tensor(np.where(pred > 0.5, 1., 0.))
loss = loss_fn(target=target, pred=prediction)
print("Accuracy: {}%".format((1. - loss)*100.))
#print("Target: {}".format(target))
#print("Pred: {}".format(prediction))



Evaluation after training:
Accuracy: 100.0%


## What are inside the model?

In [9]:
new_model.obj.state_dict()

OrderedDict([('fc1.weight',
              tensor([[ 3.6045e-02, -8.0472e-01],
                      [-5.5494e-02,  7.1364e-02],
                      [-1.7582e-01,  5.7763e-02],
                      [-2.3145e-02,  2.9227e-02],
                      [ 2.8088e-02, -2.4797e-02],
                      [ 4.0307e-02,  1.3908e-02],
                      [ 8.3561e-02, -8.5901e-02],
                      [-1.2106e-02, -6.4614e-03],
                      [ 2.7741e-02, -9.1469e-04],
                      [ 1.8237e-01,  1.2854e-01],
                      [-9.7668e-02, -1.2634e-01],
                      [-7.8994e-02, -1.6222e-01],
                      [-3.3402e-02,  2.5530e-02],
                      [ 1.1022e-01,  1.4403e-01],
                      [-1.8312e-01,  3.0588e-02],
                      [ 6.1730e-02,  1.7657e-01],
                      [-6.1536e-03, -1.0978e+00],
                      [-1.9404e-01, -1.2074e-01],
                      [ 1.5983e-01,  4.2633e-02],
                      

## Save and load model state

In [10]:
torch.save(new_model.obj.state_dict(), 'welfare.model')

In [11]:
loaded_model = Net()
loaded_model.load_state_dict(torch.load('welfare.model'))
loaded_model.eval()

Net(
  (fc1): Linear(in_features=2, out_features=20, bias=True)
  (fc2): Linear(in_features=20, out_features=10, bias=True)
  (fc3): Linear(in_features=10, out_features=1, bias=True)
)